# Image Inpainting

In this module we will describe a class of *region filling algorithms* called **image inpainting**.

Imagine finding an old family photograph. You scan it and it looks great except for a few scratches.

Of course you can load the photo in Photoshop and fix the scratches. But is that really cool? Hell no!

You are a super cool engineer! You have a reputation to live up to. You open your favorite editor and write 10 lines of code to solve the problem using an inpainting algorithm in OpenCV. If your friends do not look sufficiently impressed, you can tell them the method is based on the Navier Stokes equation they might have encountered in fluid dynamics!

But to be that cool, you need to read this module first.

## What is Image Inpainting?

Image inpainting is a class of algorithms in computer vision where the objective is to fill regions inside an image or a video.

The region is identified using a binary mask, and the filling is usually done by propagating information from the boundary of the region that needs to be filled.

The most common application of image inpainting is restoration of old scanned photos. It is also used for removing small unwanted objects in an image.

# Inpainting Algorithms
In this section, we will briefly discuss two inpainting algorithms implemented in OpenCV.

## INPAINT_NS : Navier-Stokes based Inpainting
    
This method was published in 2001 in a paper titled ["Navier-Stokes, Fluid Dynamics, and Image and Video Inpainting"](http://www.math.ucla.edu/~bertozzi/papers/cvpr01.pdf)

Sometimes I feel like the field of Computer Vision is a field of immigrants from other fields like electrical engineering, computer science, physics, and mathematics.

They bring their ideas to the field and solve the same problem in very interesting and unique ways. An electrical engineer may see an image as a 2D signal and apply the theories of signal processing to solve computer vision problems. On the other hand, a mathematician may see an image as a connected graph and solve computer vision problems using graph theory.

So it isn’t surprising that theories developed for fluid dynamics also make their way into computer vision.

In the image below, our objective is to fill the dark region and obtain an image that looks like the one on the right.

![Image Inpainting Example](https://www.learnopencv.com/wp-content/uploads/2019/04/image-inpainting-example.jpg)

How do we fill this black region? One constraint we would like is the edge entering point A should continue to the edge leaving point B. The other constraint we may want is that the region on the right of the curve joining A and B should be white, and the region on the left should be blue.

The above two constraints essentially state

1. Preserve gradients (i.e. edge like features)
2. Continue to propagate color information in smooth regions

The authors set up a partial differential equation (PDE) to update image intensities inside the region with the above constraints.

The image smoothness information is estimated by the image Laplacian and it is propagated along the isophotes (contours of equal intensities). The isophotes are estimated by the image gradient rotated by 90 degrees.

The authors show that these equations are closely related in form to the Navier-Stokes equations for 2D incompressible fluids.

The benefit of reducing the problem to one of fluid dynamics is that we benefit from well developed theoretical analysis and numerical tools.

## INPAINT_TELEA : Fast Marching Method based

This implementation is based on a paper titled ["An Image Inpainting Technique Based on the Fast Marching Method"](https://pdfs.semanticscholar.org/622d/5f432e515da69f8f220fb92b17c8426d0427.pdf) by Alexandru Telea.

This implementation solves the same constraints using a different technique. Instead of using the image Laplacian as the estimator of smoothness, the author uses a weighted average over a known image neighborhood of the pixel to inpaint. The known neighborhood pixels and gradients are used to estimate the color of the pixel to be inpainted.

Once a pixel is inpainted, the boundary needs to updated. The author treats the missing region of the image as [level sets](https://en.wikipedia.org/wiki/Level_set) and uses the [fast marching method](https://en.wikipedia.org/wiki/Fast_marching_method) to update the boundary.

### Pros and Cons

As per the theory and the papers, Navier-Stokes based inpainting is supposed to be slower and has a tendency to produce results that are blurrier than the Fast Marching based method.

In practice, we did not find that to be the case. INPAINT_NS produced better results in our tests and the speed was also marginally better than INPAINT_TELEA.

## >Inpainting Code in Python

### >Function Syntax
    
In OpenCV inpainting is implemented using the function `inpaint`.

```python
dst = cv2.inpaint(
             src, 
             inpaintMask, 
             inpaintRadius, 
             flags)
```

Where,

- **`src`** = Source image
- **`inpaintMask`** = A binary mask indicating pixels to be inpainted.
- **`dst`** = Destination image
- **`inpaintRadius`** = Neighborhood around a pixel to inpaint. Typically, if the regions to be inpainted are thin, smaller values produce better results (less blurry).
- **`flags`** : `INPAINT_NS` (Navier-Stokes based method) or `INPAINT_TELEA` (Fast marching based method)

In [ ]:
import numpy as np
import cv2 as cv


In [2]:
# OpenCV Utility Class for Mouse Handling
class Sketcher:
    def __init__(self, windowname, dests, colors_func):
        self.prev_pt = None
        self.windowname = windowname
        self.dests = dests
        self.colors_func = colors_func
        self.dirty = False
        self.show()
        cv.setMouseCallback(self.windowname, self.on_mouse)

    def show(self):
        cv.imshow(self.windowname, self.dests[0])
        cv.imshow(self.windowname + ": mask", self.dests[1])

    # onMouse function for Mouse Handling
    def on_mouse(self, event, x, y, flags, param):
        pt = (x, y)
        if event == cv.EVENT_LBUTTONDOWN:
            self.prev_pt = pt
        elif event == cv.EVENT_LBUTTONUP:
            self.prev_pt = None

        if self.prev_pt and flags & cv.EVENT_FLAG_LBUTTON:
            for dst, color in zip(self.dests, self.colors_func()):
                cv.line(dst, self.prev_pt, pt, color, 5)
            self.dirty = True
            self.prev_pt = pt
            self.show()

In [3]:
# Read image in color mode
filename = "/home/jean/Pictures/IMG_20190609_202412.jpg"
img = cv.imread(filename, cv.IMREAD_COLOR)

# If image is not read properly, return error
if img is None:
    print('Failed to load image file: {}'.format(filename))

In [4]:
# Create a copy of original image
img_mask = img.copy()
# Create a black copy of original image
# Acts as a mask
inpaintMask = np.zeros(img.shape[:2], np.uint8)
# Create sketch using OpenCV Utility Class: Sketcher
sketch = Sketcher('image', [img_mask, inpaintMask], lambda : ((255, 255, 255), 255))

In [ ]:
while True:
    ch = cv.waitKey()
    if ch == 27:
        break
    if ch == ord('t'):
        # Use Algorithm proposed by Alexendra Telea: Fast Marching Method (2004)
        # Reference: https://pdfs.semanticscholar.org/622d/5f432e515da69f8f220fb92b17c8426d0427.pdf
        res = cv.inpaint(src=img_mask, inpaintMask=inpaintMask, inpaintRadius=3, flags=cv.INPAINT_TELEA)
        cv.imshow('Inpaint Output using FMM', res)
    if ch == ord('n'):
        # Use Algorithm proposed by Bertalmio, Marcelo, Andrea L. Bertozzi, and Guillermo Sapiro: Navier-Stokes, Fluid Dynamics, and Image and Video Inpainting (2001)
        res = cv.inpaint(src=img_mask, inpaintMask=inpaintMask, inpaintRadius=3, flags=cv.INPAINT_NS)
        cv.imshow('Inpaint Output using NS Technique', res)
    if ch == ord('r'):
        img_mask[:] = img
        inpaintMask[:] = 0
        sketch.show()

In [ ]:
cv.destroyAllWindows()